In [47]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor


In [2]:
file_path = 'customer_product_price_prediction.csv'
df = pd.read_csv(file_path)
df = df.drop(df.columns[0], axis=1)

In [3]:
df.head()

,customer_id,Customer_product,Jan_amt,Feb_amt,March_amt,April_amt,May_amt,June_amt,July_amt
0,947,Travel,603,841,357,665,871,539,879
1,776,Food,877,977,28,303,351,760,145
2,932,Electronics,567,843,364,546,780,348,403
3,364,Electronics,749,858,21,414,167,569,347
4,397,Pet Care,8,71,926,554,53,227,675


In [4]:
df.Customer_product.value_counts()

Customer_product
Grocery               879
Travel                867
Education             854
Family                841
Food                  835
Electronics           826
Pet Care              826
Home Improvement      823
Movies                821
Shopping              820
Sports and Fitness    809
Health and Beauty     799
Name: count, dtype: int64

In [8]:
#lets first prepare the data in product_amt pair for each customer

In [14]:
data_grouped = df.groupby(['customer_id', 'Customer_product']).sum().reset_index()
data_grouped.head()



,customer_id,Customer_product,Jan_amt,Feb_amt,March_amt,April_amt,May_amt,June_amt,July_amt
0,1,Education,1017,2691,867,836,1509,2037,833
1,1,Family,1827,1112,1146,1271,1161,1308,1092
2,1,Food,865,522,510,24,363,257,842
3,1,Grocery,584,895,1566,1519,1684,442,1236
4,1,Home Improvement,1253,1678,727,2353,1715,1108,1080


In [15]:
pivot_data = data_grouped.pivot(index='customer_id', columns='Customer_product', values=['Jan_amt', 'Feb_amt', 'March_amt', 'April_amt', 'May_amt', 'June_amt', 'July_amt'])
pivot_data.head()

Jan_amt                                     \
Customer_product Education Electronics  Family   Food Grocery   
customer_id                                                     
1                   1017.0         NaN  1827.0  865.0   584.0   
2                   1390.0       567.0     NaN  965.0     NaN   
3                      NaN         NaN   803.0   71.0   287.0   
4                      NaN         NaN     NaN    NaN  1383.0   
5                    866.0      1985.0   790.0  610.0     NaN   

                                                                              \
Customer_product Health and Beauty Home Improvement Movies Pet Care Shopping   
customer_id                                                                    
1                              NaN           1253.0  784.0    787.0    737.0   
2                            332.0              NaN    NaN      NaN   1117.0   
3                              NaN              NaN    NaN      NaN   1566.0   
4                              NaN              NaN    NaN      NaN      NaN   
5                            950.0              NaN    NaN      NaN    685.0   

                  ... July_amt                                   \
Customer_product  ...   Family   Food Grocery Health and Beauty   
customer_id       ...                                             
1                 ...   1092.0  842.0  1236.0               NaN   
2                 ...      NaN  736.0     NaN             197.0   
3                 ...    226.0  468.0    16.0               NaN   
4                 ...      NaN    NaN  1271.0               NaN   
5                 ...   1438.0  113.0     NaN             255.0   

                                                                               \
Customer_product Home Improvement Movies Pet Care Shopping Sports and Fitness   
customer_id                                                                     
1                          1080.0  965.0    709.0    292.0                NaN   
2                             NaN    NaN      NaN    836.0                NaN   
3                             NaN    NaN      NaN   1858.0                NaN   
4                             NaN    NaN      NaN      NaN             1330.0   
5                             NaN    NaN      NaN     20.0              258.0   

                          
Customer_product  Travel  
customer_id               
1                    NaN  
2                  661.0  
3                  421.0  
4                    NaN  
5                 1814.0  

[5 rows x 84 columns]

In [16]:
pivot_data.columns = [f'{col[1].lower()}_{col[0].split("_")[0].lower()}_amount' for col in pivot_data.columns]

In [17]:
pivot_data.head()

,education_jan_amount,electronics_jan_amount,family_jan_amount,food_jan_amount,grocery_jan_amount,health and beauty_jan_amount,home improvement_jan_amount,movies_jan_amount,pet care_jan_amount,shopping_jan_amount,...,family_july_amount,food_july_amount,grocery_july_amount,health and beauty_july_amount,home improvement_july_amount,movies_july_amount,pet care_july_amount,shopping_july_amount,sports and fitness_july_amount,travel_july_amount
customer_id,,,,,,,,,,,,,,,,,,,,,
1,1017.0,NaN,1827.0,865.0,584.0,NaN,1253.0,784.0,787.0,737.0,...,1092.0,842.0,1236.0,NaN,1080.0,965.0,709.0,292.0,NaN,NaN
2,1390.0,567.0,NaN,965.0,NaN,332.0,NaN,NaN,NaN,1117.0,...,NaN,736.0,NaN,197.0,NaN,NaN,NaN,836.0,NaN,661.0
3,NaN,NaN,803.0,71.0,287.0,NaN,NaN,NaN,NaN,1566.0,...,226.0,468.0,16.0,NaN,NaN,NaN,NaN,1858.0,NaN,421.0
4,NaN,NaN,NaN,NaN,1383.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1271.0,NaN,NaN,NaN,NaN,NaN,1330.0,NaN
5,866.0,1985.0,790.0,610.0,NaN,950.0,NaN,NaN,NaN,685.0,...,1438.0,113.0,NaN,255.0,NaN,NaN,NaN,20.0,258.0,1814.0


In [20]:
pivot_data.reset_index(inplace=True)

In [21]:
pivot_data.head(5)

,customer_id,education_jan_amount,electronics_jan_amount,family_jan_amount,food_jan_amount,grocery_jan_amount,health and beauty_jan_amount,home improvement_jan_amount,movies_jan_amount,pet care_jan_amount,...,family_july_amount,food_july_amount,grocery_july_amount,health and beauty_july_amount,home improvement_july_amount,movies_july_amount,pet care_july_amount,shopping_july_amount,sports and fitness_july_amount,travel_july_amount
0,1,1017.0,NaN,1827.0,865.0,584.0,NaN,1253.0,784.0,787.0,...,1092.0,842.0,1236.0,NaN,1080.0,965.0,709.0,292.0,NaN,NaN
1,2,1390.0,567.0,NaN,965.0,NaN,332.0,NaN,NaN,NaN,...,NaN,736.0,NaN,197.0,NaN,NaN,NaN,836.0,NaN,661.0
2,3,NaN,NaN,803.0,71.0,287.0,NaN,NaN,NaN,NaN,...,226.0,468.0,16.0,NaN,NaN,NaN,NaN,1858.0,NaN,421.0
3,4,NaN,NaN,NaN,NaN,1383.0,NaN,NaN,NaN,NaN,...,NaN,NaN,1271.0,NaN,NaN,NaN,NaN,NaN,1330.0,NaN
4,5,866.0,1985.0,790.0,610.0,NaN,950.0,NaN,NaN,NaN,...,1438.0,113.0,NaN,255.0,NaN,NaN,NaN,20.0,258.0,1814.0


In [34]:
data = pivot_data.fillna(pivot_data.median())

In [35]:
data.isnull().sum()

customer_id                       0
education_jan_amount              0
electronics_jan_amount            0
family_jan_amount                 0
food_jan_amount                   0
                                 ..
movies_july_amount                0
pet care_july_amount              0
shopping_july_amount              0
sports and fitness_july_amount    0
travel_july_amount                0
Length: 85, dtype: int64

In [37]:
# Now lets try to do modelling
# In our case we have more than one predictor variable, so we need to use Multioutput Regression

# Multi-output Regression
# We know in simple regression we predict a single output variable but where
# the goal is to predict multiple target variables instead of just one we use Multi-output regression.
# Multi-output regression is mostly used for addressing complex real-world problems.
# This approach is particularly useful when variables are interrelated or when it is more efficient to 
# make joint predictions as the prediction of one variable can affect the prediction of other variables, 
# we’ll understand this thing in the subtopic of chained regression.

# y1, y2,....ym = f(x1,x2,.....xn)
# where:

# y1, y2, …, ym are the output variables
# x1, x2, …, xn are the input variables
# f is the regression function

In [46]:
target_cols = [x for x in data.columns if x.endswith('_june_amount')] + [x for x in data.columns if x.endswith('_july_amount')]
target_cols


['education_june_amount',
 'electronics_june_amount',
 'family_june_amount',
 'food_june_amount',
 'grocery_june_amount',
 'health and beauty_june_amount',
 'home improvement_june_amount',
 'movies_june_amount',
 'pet care_june_amount',
 'shopping_june_amount',
 'sports and fitness_june_amount',
 'travel_june_amount',
 'education_july_amount',
 'electronics_july_amount',
 'family_july_amount',
 'food_july_amount',
 'grocery_july_amount',
 'health and beauty_july_amount',
 'home improvement_july_amount',
 'movies_july_amount',
 'pet care_july_amount',
 'shopping_july_amount',
 'sports and fitness_july_amount',
 'travel_july_amount']

In [51]:
features = data.drop(columns=target_cols)
target = data[target_cols]
target

,education_june_amount,electronics_june_amount,family_june_amount,food_june_amount,grocery_june_amount,health and beauty_june_amount,home improvement_june_amount,movies_june_amount,pet care_june_amount,shopping_june_amount,...,family_july_amount,food_july_amount,grocery_july_amount,health and beauty_july_amount,home improvement_july_amount,movies_july_amount,pet care_july_amount,shopping_july_amount,sports and fitness_july_amount,travel_july_amount
0,2037.0,701.0,1308.0,257.0,442.0,740.0,1108.0,865.0,523.0,234.0,...,1092.0,842.0,1236.0,706.0,1080.0,965.0,709.0,292.0,668.0,685.0
1,1740.0,793.0,715.0,400.0,707.0,189.0,671.0,680.0,668.0,811.0,...,685.0,736.0,720.5,197.0,651.5,599.0,658.0,836.0,668.0,661.0
2,675.0,701.0,881.0,251.0,126.0,740.0,671.0,680.0,668.0,2088.0,...,226.0,468.0,16.0,706.0,651.5,599.0,658.0,1858.0,668.0,421.0
3,675.0,701.0,715.0,700.0,1748.0,740.0,671.0,680.0,668.0,642.0,...,685.0,688.5,1271.0,706.0,651.5,599.0,658.0,645.0,1330.0,685.0
4,390.0,2178.0,1307.0,624.0,707.0,570.0,671.0,680.0,668.0,534.0,...,1438.0,113.0,720.5,255.0,651.5,599.0,658.0,20.0,258.0,1814.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,675.0,2835.0,2299.0,700.0,707.0,980.0,671.0,548.0,224.0,684.0,...,1283.0,688.5,720.5,127.0,651.5,3.0,394.0,157.0,668.0,685.0
996,675.0,1069.0,715.0,894.0,385.0,550.0,1077.0,499.0,668.0,642.0,...,685.0,1980.0,702.0,773.0,1468.0,833.0,658.0,645.0,1069.0,685.0
997,675.0,382.0,715.0,441.0,479.0,1862.0,671.0,111.0,668.0,642.0,...,685.0,27.0,1525.0,902.0,651.5,670.0,658.0,645.0,565.0,685.0
998,864.0,701.0,715.0,700.0,707.0,240.0,1796.0,680.0,668.0,642.0,...,685.0,688.5,720.5,424.0,949.0,599.0,658.0,645.0,668.0,856.0


In [52]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [53]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [54]:
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train_scaled, y_train)

MultiOutputRegressor(estimator=LinearRegression())

In [55]:
y_pred = model.predict(X_test_scaled)

In [59]:
df_result = pd.DataFrame(y_pred)

In [60]:
df_result.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,730.074537,778.106093,665.880954,722.929825,633.280673,789.064137,682.923504,637.136104,416.869291,346.057431,...,733.001944,590.449181,758.484098,725.636361,566.072768,409.618574,384.816469,343.353036,726.747994,559.963856
1,661.980099,546.700636,653.928463,630.347631,688.658410,680.886369,517.855389,696.512088,654.410209,601.606952,...,681.420180,594.364739,717.201582,748.397456,455.132377,582.919958,614.161121,633.485603,687.232773,494.424866
2,472.768107,545.378632,862.221999,1293.990781,662.799482,746.767021,621.311935,490.768086,858.703622,812.155736,...,733.601899,1404.612785,693.653579,592.758923,569.132998,458.805808,696.391577,681.929720,677.072934,591.659488
3,1383.758812,679.479117,638.360421,627.147833,792.316561,788.332743,704.937436,590.640995,490.896461,555.850230,...,760.084621,682.252043,803.288725,689.900911,797.486988,659.785404,486.560953,591.133795,673.474073,864.478121
4,420.554821,744.719617,723.321139,869.091409,620.118414,728.142558,738.633307,722.454352,758.677074,560.282636,...,722.763004,869.710206,690.058075,727.144908,669.379704,583.954687,713.049190,602.313578,684.294345,689.957962


In [57]:
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='raw_values')

print(f'{mse[0]}')
print(f'{r2[0]}')
print(f'{mse[1]}')
print(f'{r2[1]}')

93208.99928022575
0.3511487322099025
82691.62503384208
0.367572264656308
